# Linear Transformation 
We will start with the alignement of the same models for different seeds. 
- First we do not restrict the matrix.
- Second we restrict the matrix to be a rotation matrix.
- Thrid use affine transformation 
- Last but not least we are using different norms and regularization techniques to improve the results



Steps: 
- Load the same model but with different seed
- Sample different images and get latent representation 
- Create Datamatrix X and X'
- Solve the simple optimization problem (Using closed form solution as well cvxpy solver)


In [1]:
# Import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
import cvxpy as cp

In [2]:
# Configuration

config = {
    'path1': "/Users/mariotuci/Desktop/Google-Drive/Master/SoSe-24/Project Studies/Project/Code/latent-communication/VAE/models/MNIST_VAE_3_10.pth",
    'modelname1': 'VAE',
    'seed1': '3',
    'path2': "/Users/mariotuci/Desktop/Google-Drive/Master/SoSe-24/Project Studies/Project/Code/latent-communication/VAE/models/MNIST_VAE_4_10.pth",
    'modelname2': 'VAE',
    'seed2': '4',
    'num_samples': '100',
    'storage_path': 'VAE-LinearTransform',

}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transdormations
transformations = [transforms.ToTensor(), 
                                # Normalize between -1 and 1
                                transforms.Normalize((0.5,), (0.5,)),
                                # Flatten the Image to a vector
                                transforms.Lambda(lambda x: x.view(-1) )
                                ]




# Optimization Problem in the Linear Case 
Let $x^i,y^i \in \mathbb{R^n}$ for $i = 1,...,m$ and $A \in \mathbb{R}^{n \times n}$ we are looking for the optimal A, which solves the following optimization problem 
$$ min_A \sum_{i = 1}^n ||Ax^i - y^i||^2 $$
$$ min_ a \sum_{i=1} \sum_{j =1} (A_{(j)} x^i - y^i_j)^2 $$
where we are using the euclidian norm when not otherwise stated.



## Load Model and Transformed Data for VAE
In this section we load the trained models, which we prepaired for our experimental setup

In [6]:
import os
# Change directory
os.chdir('/Users/mariotuci/Desktop/Google-Drive/Master/SoSe-24/Project Studies/Project/Code/latent-communication')

from helper.DataLoaderMNIST import DataLoader_MNIST  # Import the data loader


def load_model(model_name, model_path):
    DEVICE = torch.device('cpu')
    
    if model_name == 'VAE':
        from VAE.model_def import VAE
        model = VAE(in_dim=784, dims=[256, 128, 64, 32], distribution_dim=16).to(DEVICE)
    elif model_name == 'resnet':
        from resnet.model_def import ResNet
        model = ResNet().to(DEVICE)
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    return model

def load_Models():
    model1 = load_model(config['modelname1'], config['path1'])
    model2 = load_model(config['modelname2'], config['path2'])
    return model1, model2

def get_transformations(model_name):
    if model_name == 'VAE':
        return [
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
            transforms.Lambda(lambda x: x.view(-1))
        ]
    elif model_name == 'resnet':
        return [
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x.repeat(3, 1, 1))
        ]
    else:
        raise ValueError(f"Unknown model name: {model_name}")

def transformations():
    transformations1 = get_transformations(config['modelname1'])
    transformations2 = get_transformations(config['modelname2'])
    return transformations1, transformations2

# Load models
model1, model2 = load_Models()

# Get transformations
transformations1, transformations2 = transformations()

# Initialize data loader
data_loader = DataLoader_MNIST(128, transformations1, transformations2)

# Get train and test loaders
train_loader1, train_loader2 = data_loader.get_train_loader()
test_loader1, test_loader2 = data_loader.get_test_loader()





# Sample from Dataset

In [8]:
from utils.sampler import simple_sampler, class_sampler

# Convertiere String in Integer
num_samples= int(config['num_samples'])

print(num_samples)
# Sample simply 
z1, z2= simple_sampler(num_samples, model1, model2, data_loader, DEVICE)
# Sample from each class
#z1_suf, z2_suf, images_suf, labels_suf = class_sampler(config['num_samples'], model, model2, data_loader, DEVICE)



100


AttributeError: 'DataLoader_MNIST' object has no attribute 'train_loader'

# Convex Optimization Solver
## Linear Transformation with regularization
We are using the cvxpy solver which is a open source solver for convex optimization problems.

In [88]:
import os 
#ä Get wd 
print(os.getcwd())
# Linear Transformation
from optimizer import LinearFitting

linear_fitting = LinearFitting(z1, z2,lamda=0.01)

linear_fitting.solve_problem()

loss, A  = linear_fitting.get_results()

name = 'Linear_' + config['modelname1'] + '_' + config['seed1'] + '_' + config['modelname2'] + '_' + config['seed2'] + '_' + config['num_samples']
path = 'optimization/VAE-LinearTransform/' + name 

linear_fitting.save_results(path)



/Users/mariotuci/Desktop/Google-Drive/Master/SoSe-24/Project Studies/Project/Code/latent-communication
Solving the problem
Defining the problem
Results saved at  optimization/VAE-LinearTransform/Linear_VAE_3_VAE_4_100


## Affine transformation
In this section we implement the affine transformation. We are adding a offset to the problem.

In [89]:
from optimizer import AffineFitting

affine_fitting = AffineFitting(z1, z2, lamda=0.01)

# Solve the problem
affine_fitting.solve_problem()

# Get the results
loss, A, b = affine_fitting.get_results()

# Show the results
print('Loss:', loss)
print('A:', A)
print('b:', b)

# Save the results
name = 'Affine_' + config['modelname1'] + '_' + config['seed1'] + '_' + config['modelname2'] + '_' + config['seed2'] + '_' + config['num_samples']
path = 'optimization/VAE-LinearTransform/' + name


# Save the results
affine_fitting.save_results(path)



Solving the problem
Defining the problem
Getting results
Loss: 70.35849794230826
A: [[-5.93303879e-01 -3.06867819e-01  0.00000000e+00 ... -2.42000117e-01
  -1.52473218e-01 -7.68865392e-02]
 [-3.01676852e-01 -1.00814253e-01  0.00000000e+00 ... -5.01366241e-02
  -7.06784603e-02 -4.19766510e-02]
 [ 3.13299623e-01  1.58261536e-01  0.00000000e+00 ...  6.37404351e-02
   6.02800941e-02  8.09385848e-02]
 ...
 [ 4.19908121e-01  3.04512863e-01  0.00000000e+00 ...  6.81596693e-02
   1.04791855e-01  9.78241610e-02]
 [ 2.11636623e-02  5.12156567e-03  0.00000000e+00 ...  5.70270631e-03
   2.73593204e-03  5.67791150e-03]
 [-8.23454890e-02  1.36501746e-01  0.00000000e+00 ...  7.76349126e-02
  -3.89713996e-04 -3.83569196e-02]]
b: [ 3.52062489e-01  3.76244028e+00  5.29228343e-01  3.44155825e+00
  3.14042208e-13  3.10253037e+00  1.41707738e+00  2.35135171e+00
 -2.21233827e-13  2.11076399e+00 -5.62412252e-14  2.11808798e+00
 -5.47060826e-14  3.81549238e+00  4.64853700e-15  3.92733825e+00
  8.28841365e-01 

## Linear Transformation with constraints (psd)
In this section we relax the problem and consider that the matrix has to be positive semidefinite.

In [47]:
# Optimization Variable
A_psd = cp.Variable((32, 32))

# Loss Function
loss_psd = cp.norm2(cp.vstack([A_psd @ z1[i] - z2[i] for i in range(z1.shape[0])]))**2 + lamda * cp.norm(A_psd, 'fro')**2

# Objective Function
objective_psd = cp.Minimize(loss_psd)

# Constraints
constraints = [A_psd >> 0]

# Problem

problem_psd = cp.Problem(objective_psd, constraints)

# Solve the problem

problem_psd.solve()

# Print results
print("Optimal value: ", problem_psd.value)
print(A_psd.value)



Optimal value:  34.901493456700365
[[ 1.51821848e-01  5.56895851e-02 -1.93031590e-02 ...  3.39359862e-02
   5.35622009e-02  8.20711810e-02]
 [ 2.08683234e-02  8.40538183e-02 -9.98856392e-04 ...  1.12780196e-02
   1.49802143e-01 -8.53980783e-02]
 [ 6.25820417e-02  2.61333410e-02  4.37348532e-02 ... -9.39537026e-03
  -3.47203346e-02  1.30371568e-02]
 ...
 [ 4.75892774e-02  7.40579969e-02  2.52987499e-02 ...  8.17855467e-02
  -8.75878002e-02  1.58779796e-01]
 [-4.74974407e-02 -7.88330682e-02  2.08291555e-03 ...  6.51295726e-02
   5.02004975e-02 -6.86589407e-02]
 [-2.99752488e-02  2.91760625e-01  1.07695391e-02 ... -8.81125805e-05
   1.24254865e-01  1.52271000e-01]]


## Affine Transfomrmation with constraint (psd)

In [48]:
A_aff_psd = cp.Variable((32, 32))
b_aff_psd = cp.Variable(32)

# Loss Function
loss_aff_psd = cp.norm2(cp.vstack([A_aff_psd @ z1[i] + b_aff_psd - z2[i] for i in range(z1.shape[0])]))**2 + lamda * cp.norm(A_aff_psd, 'fro')**2

# Objective Function
objective_aff_psd = cp.Minimize(loss_aff_psd)

# Constraints
constraints_aff_psd = [A_aff_psd >> 0]

# Problem
problem_aff_psd = cp.Problem(objective_aff_psd, constraints_aff_psd)

# Solve the problem
problem_aff_psd.solve()

# Print results
print("Optimal value: ", problem_aff_psd.value)
print(A_aff_psd.value)
print(b_aff_psd.value)


Optimal value:  34.24464274665315
[[ 0.11666089  0.00393495 -0.0056867  ...  0.0191255   0.0107472
   0.04334259]
 [-0.02276971  0.04896931  0.02793243 ...  0.00659383  0.10956296
  -0.0971355 ]
 [ 0.00696019 -0.04730342  0.03268734 ... -0.004706   -0.05826982
  -0.02599201]
 ...
 [ 0.02691977  0.04183781  0.01845002 ...  0.06482117 -0.11482373
   0.13552223]
 [-0.03337478 -0.05176091  0.01215324 ...  0.07089855  0.05565875
  -0.03461887]
 [-0.10156949  0.20056511  0.00328381 ... -0.02638674  0.06216649
   0.09022269]]
[ 2.34381618  2.31231468  1.91114035  4.66982296  1.3469774   1.59156096
  3.91057957  3.30460911  0.8458655   1.51884706 -0.6437429   2.43651978
 -0.03508159  2.62580291 -1.27235726  1.47504172  0.04440307  0.27604265
 -0.82264772 -0.3927284   3.35220021  1.87762301  1.13077015  1.21421183
  2.84487957  3.69182049  1.14590623  6.95825472  0.15937158  1.61406641
 -0.64105622  4.57345625]


## Trying different norms 
We begin to reformulate the problem with the L1 Norm



In [53]:
A_L1 = cp.Variable((32, 32))

#May need another solver



# Loss Function
loss_L1 = cp.norm1(cp.vstack([A_L1 @ z1[i] - z2[i] for i in range(z1.shape[0])]))**2 + lamda * cp.norm(A_L1, '')**2

# Objective Function
objective_L1 = cp.Minimize(loss_L1)


# Problem
problem_L1= cp.Problem(objective_L1)

# Solve the problem
problem_L1.solve(verbose=True, solver=cp.ECOS)

# Print results
print("Optimal value: ", problem_L1.value)
print(A_L1.value)


                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) May 24 10:33:27 PM: Your problem has 1024 variables, 0 constraints, and 0 parameters.
(CVXPY) May 24 10:33:27 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 24 10:33:27 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 24 10:33:27 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 24 10:33:27 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 24 10:33:27 PM: Compiling problem (target solver=ECOS).
(C

(CVXPY) May 24 10:33:27 PM: Applying reduction Dcp2Cone
(CVXPY) May 24 10:33:27 PM: Applying reduction CvxAttr2Constr
(CVXPY) May 24 10:33:27 PM: Applying reduction ConeMatrixStuffing
(CVXPY) May 24 10:33:27 PM: Applying reduction ECOS
(CVXPY) May 24 10:33:27 PM: Finished problem compilation (took 1.549e-01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) May 24 10:33:27 PM: Invoking solver ECOS  to obtain a solution.

ECOS 2.0.10 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -2.913e-01  +9e+03  5e-01  6e-01  1e+00  1e+00    ---    ---    1  1  - |  -  - 
 1  +5.972e+00  +7.520e+00  +3e+03  8e-01  6e-01  3e+00  4e-01  0.7231  2e-02   1  1  

/Users/mariotuci/miniconda/envs/relreps/lib/python3.8/site-packages/cvxpy/reductions/solvers/solving_chain.py:354: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add an explicit dependency on ECOS or switch to our new
    default solver, Clarabel, by either not specifying a solver argument
    or specifying ``solver=cp.CLARABEL``.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


 5  +3.506e+02  +3.715e+02  +2e+02  1e+00  7e-01  2e+01  3e-02  0.6777  4e-01   1  1  1 |  0  0
 6  +1.176e+03  +1.206e+03  +1e+02  5e-01  3e-01  3e+01  2e-02  0.8604  4e-01   2  1  1 |  0  0
 7  +2.806e+02  +4.176e+02  +7e+01  1e+00  6e-01  1e+02  1e-02  0.6592  6e-01   2  1  1 |  0  0
 8  +1.565e+03  +1.565e+03  +7e+01  5e-01  3e-01  6e-01  1e-02  0.4537  8e-01   2  1  1 |  0  0
 9  +4.330e+03  +4.332e+03  +1e+01  2e-01  1e-01  2e+00  2e-03  0.7977  2e-02   1  1  1 |  0  0
10  +8.064e+03  +8.076e+03  +1e+01  1e-01  8e-02  1e+01  1e-03  0.5704  4e-01   1  1  1 |  0  0
11  +5.106e+03  +5.150e+03  +7e+00  3e-01  9e-02  4e+01  1e-03  0.5788  6e-01   2  2  1 |  0  0
12  +1.541e+04  +1.545e+04  +5e+00  9e-02  4e-02  4e+01  7e-04  0.9890  6e-01   1  2  1 |  0  0
13  +2.494e+04  +2.497e+04  +2e+00  5e-02  2e-02  4e+01  3e-04  0.7629  2e-01   1  2  2 |  0  0
14  +3.710e+04  +3.712e+04  +7e-01  2e-02  9e-03  2e+01  1e-04  0.6932  2e-01   3  2  2 |  0  0
15  +3.486e+04  +3.489e+04  +7e-01  3e-0

/Users/mariotuci/miniconda/envs/relreps/lib/python3.8/site-packages/cvxpy/problems/problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


# Compare the total distance 

In [54]:
# Get Latent Space for al

NameError: name 'installed_solvers' is not defined